In [ ]:
#same as ELO.ipynb but for Womens
import pandas as pd
import numpy as np
import math
from sklearn.metrics import brier_score_loss
from collections import defaultdict
import warnings
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)
warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
def def_elo():
    return 1500
tournamentchange = {"NCAA": 1.5, "NIT": 1.3, "CBI": 1.2, "NIT ": 1.3, "V16": 1.1, "TBC": 1.1, "CIT": 1.1, "Regular":1}


In [ ]:
regularseason = pd.read_csv("march-machine-learning-mania-2024/WRegularSeasonCompactResults.csv")
postseason = pd.read_csv("march-machine-learning-mania-2024/WNCAATourneyCompactResults.csv")
#conferencegames = pd.read_csv("march-machine-learning-mania-2024/WConferenceTourneyGames.csv")
#secondary = pd.read_csv("march-machine-learning-mania-2024/WSecondaryTourneyCompactResults.csv")
conferences = pd.read_csv("march-machine-learning-mania-2024/WTeamConferences.csv")
regularseason["scorediff"] = regularseason["WScore"] - regularseason["LScore"]
postseason["scorediff"] = postseason["WScore"] - postseason["LScore"]
#secondary["scorediff"] = secondary["WScore"] - secondary["LScore"]

In [ ]:
postseason["Tournament"] = "NCAA"
#secondary["Tournament"] =  secondary["SecondaryTourney"]
regularseason["Tournament"] = "Regular"
#conferencegames["Tournament"] = "Conference"

In [ ]:
allgames = pd.concat([postseason, regularseason])

In [ ]:
allgames["Tournament"].unique()
allgames.iloc[allgames["Tournament"].map(tournamentchange).argsort()]

In [ ]:
def expected_score(eloa, elob, isHome, home_advantage):
    """
    Gets the expected probabilty of winning for team A against team B
    eloa: the ELO of team A
    elob: the ELO of team B
    isHome: number which describes whether A is home(1), neutral(0), or away(-1)
    home_advantage: home advantage quantified in added winning odds
    """
    #from eloa pov
    expecta = 1/(1 + 10**((elob - eloa)/400)) #calculates expected
    homeadd = expecta + home_advantage
    awayloss = expecta - home_advantage
    if homeadd > 1: #if more than 1 or less than 0, set to 1 and 0, respectively(edge case)
        homeadd = 1
    if awayloss < 0:
        awayloss = 0
    
    if isHome == 1:
        return homeadd
    elif isHome == 0:
        return expecta
    else:
        return awayloss

In [ ]:
def elo_change(eloa, elob, scalar, mov, win, expected_score, mov_factor, tournament):
    """
    Determines the change in elo after a result
    """
    if win == 1:
        winelodiff = eloa - elob
    else:
        winelodiff = elob - eloa
    mov_factor = (math.log(mov, 10) + 1) * (2.2) * (mov_factor)/((winelodiff) * .001 + 2.2)
    change = scalar * mov_factor * (win - expected_score) * tournamentchange[tournament]
    return change

In [ ]:
def clean_Home(WLoc):
    if WLoc == "H":
        return 1
    elif WLoc == "N":
        return 0
    else:
        return -1

In [ ]:
regularseason["WLoc_Num"] = regularseason["WLoc"].apply(clean_Home)
postseason["WLoc_Num"] = postseason["WLoc"].apply(clean_Home)
allgames["WLoc_Num"] = allgames["WLoc"].apply(clean_Home)

In [ ]:
def set_zero(df, cols):
    for col in cols:
        df[col] = 0
colchange = ["WinnerElo", "LoserElo", "WinningOdds", "WChange", "LChange"]

In [ ]:
set_zero(regularseason, colchange)
set_zero(postseason, colchange)
set_zero(allgames, colchange)

In [ ]:
def simulate_game(elos, df, home_advantage):
    for game in range(df.shape[0]):
        winner = df["WTeamID"][game]
        loser = df["LTeamID"][game]
        winner_elo = elos[winner]
        loser_elo = elos[loser]
        df["WinnerElo"][game] = float(winner_elo)
        df["LoserElo"][game] = float(loser_elo)
        game_expected_score = expected_score(winner_elo, loser_elo, df["WLoc_Num"][game], home_advantage)
        change = elo_change(winner_elo, loser_elo, scalar, df["scorediff"][game], 1, game_expected_score, mov_factor, df["Tournament"][game])
        df["WChange"][game] = change
        df["LChange"][game] = -1 * change
        elos[winner] = elos[winner] + change
        elos[loser] = elos[loser] - change
        df["WinningOdds"][game] = game_expected_score

In [ ]:
def get_conference(year, team):
    conf = conferences[(conferences["Season"] == year) & (conferences["TeamID"] == team)].reset_index()
    if conf.shape[0] >= 1:
        return conf["ConfAbbrev"][0]
    else:
        return "none"
get_conference(2011, 3709)

In [ ]:
import time
def get_variance(iqrs, vars, means, teams, df):
    for team in teams:
        Wgames = df[df["WTeamID"] == team]
        Lgames = df[df["LTeamID"] == team]
        changes = pd.concat([Wgames["WChange"], Lgames["LChange"]], axis = 0)
        q1 = changes.quantile(0.25)
        q3 = changes.quantile(0.75)
        iqrs[team] = q3 - q1
        vars[team] = changes.std()
        means[team] = changes.median()
def apply_variance(iqrs, vars, means, df):
    for game in range(df.shape[0]):
        df["WIQR"][game] = iqrs[df["WTeamID"][game]]
        df["LIQR"][game] = iqrs[df["LTeamID"][game]]
        df["WSD"][game] = vars[df["WTeamID"][game]]
        df["LSD"][game] = vars[df["LTeamID"][game]]
        df["WMedian"][game] = means[df["WTeamID"][game]]
        df["LMedian"][game] = means[df["LTeamID"][game]]

In [ ]:
scalar = 10
mov_factor = 1
difference = 0.1
from copy import copy
def test_elo(scalar, mov_factor):
    count = 0
    brier_scores = []
    no_tournyelos = defaultdict(def_elo)
    elodf = pd.DataFrame()
    alltourneyelodf = pd.DataFrame()
    notourneyelodf = pd.DataFrame()
    global_iqrs = {}
    global_vars = {}
    global_means = {}
    for seasons in pd.unique(regularseason["Season"]):
        #print(seasons)
        season = regularseason[regularseason["Season"] == seasons].reset_index()
        tournament = postseason[postseason["Season"] == seasons].reset_index()
        wholeseason = allgames[allgames["Season"] == seasons].reset_index()
        wholeseason = wholeseason.iloc[wholeseason["Tournament"].map(tournamentchange).argsort()]
        home_winning = season[season["WLoc_Num"] == 1].shape[0] / (season[season["WLoc_Num"] == 1].shape[0] + season[season["WLoc_Num"] == -1].shape[0])
        home_advantage = home_winning - 0.5 
        tournament["True"] = 1
        tournyelos = copy(no_tournyelos)
        seasonteams = pd.concat([season["WTeamID"], season["LTeamID"]], axis = 0)
        simulate_game(no_tournyelos, season, home_advantage)
        simulate_game(tournyelos, wholeseason, home_advantage)
        get_variance(global_iqrs, global_vars, global_means, seasonteams.unique(), season)
        pretournament = copy(no_tournyelos)
        set_zero(tournament, ["WSD", "WIQR", "LSD", "LIQR", "WMedian", "LMedian"])
        
        if tournament.shape[0] > 0:
            iqrs = {}
            vars = {}
            means = {}
            teams = pd.concat([tournament["WTeamID"], tournament["LTeamID"]], axis = 0)
            get_variance(iqrs, vars, means, teams.unique(), season)
            apply_variance(iqrs, vars, means, tournament)
            simulate_game(no_tournyelos, tournament, home_advantage)
            brier_scores.append(brier_score_loss(tournament["True"], tournament["WinningOdds"]))
            elodf = pd.concat([elodf, tournament], ignore_index = True)
        alltourneyelodf = pd.concat([alltourneyelodf, wholeseason], ignore_index = True)
        notourneyelodf = pd.concat([notourneyelodf, season])
        conferencemeans = {}
        conferencemeans["none"] = 1500
        no_tournyelos = copy(tournyelos)
        
        for key in no_tournyelos:
            #print(seasons, key)
            keyconf = get_conference(seasons, key)
            if keyconf not in conferencemeans and keyconf != "none":
                conferenceteams = conferences[(conferences["ConfAbbrev"] == keyconf) & (conferences["Season"] == seasons)].reset_index()
                conferenceelos = []
                for team in conferenceteams["TeamID"].unique():
                    conferenceelos.append(no_tournyelos[team])
                conferencemeans[keyconf] = np.mean(conferenceelos)
            conferencemean = conferencemeans[keyconf]
            if no_tournyelos[key] > conferencemean:
                no_tournyelos[key] = no_tournyelos[key] - (abs(no_tournyelos[key] - conferencemean))/4
            elif no_tournyelos[key] < conferencemean:
                no_tournyelos[key] = no_tournyelos[key] + (abs(no_tournyelos[key] - conferencemean))/4
    #return np.mean(brier_scores), [scalar, mov_factor, np.mean(brier_scores)]
    return elodf, alltourneyelodf, notourneyelodf, no_tournyelos, pretournament, global_iqrs, global_vars, global_means


In [ ]:
elodf, alltourneyelodf, notourneyelodf, tournyelos, pretournament, global_iqrs,global_vars, global_means = test_elo(4.2, 5.8) 

#output.to_csv("ELO.csv", index = False)

In [ ]:
max(tournyelos)

In [ ]:
from heapq import nlargest
nlargest(10, tournyelos, key=tournyelos.get)
#max(tournyelos, key=tournyelos.get)

In [ ]:
difference = 1
smallest_brier = 1
smallest_coord = []
coordinates = []
for scalar in np.arange(4.1, 4.3, 0.01):
    for mov_factor in np.arange(5.7, 5.9, 0.01):
        avg_brier, coord = test_elo(scalar, mov_factor)
        if avg_brier < smallest_brier:
            smallest_brier = avg_brier
            smallest_coord = coord
        coordinates.append(coord)
        print(avg_brier, coord)
print(smallest_brier)
print(smallest_coord)
print(coordinates) #scalar = 4.2, mov_factor = 5.8


In [ ]:
elos = pd.DataFrame.from_dict(tournyelos,orient='index', columns=['ELO'])
elos.index.name = "Team"
elos = elos.reset_index()

In [ ]:
elos_vars = pd.DataFrame.from_dict(global_vars,orient='index', columns=['VARS'])
elos_vars.index.name = "Team"
elos_vars = elos_vars.reset_index()

In [ ]:
elos_iqr = pd.DataFrame.from_dict(global_iqrs,orient='index', columns=['IQR'])
elos_iqr.index.name = "Team"
elos_iqr = elos_iqr.reset_index()

In [ ]:
elos_means = pd.DataFrame.from_dict(global_means,orient='index', columns=['MEAN'])
elos_means.index.name = "Team"
elos_means = elos_means.reset_index()

In [ ]:
elocombined = pd.merge(elos, elos_vars, on = "Team")
elocombined = pd.merge(elocombined, elos_means, on = "Team")
elocombined = pd.merge(elocombined, elos_iqr, on = "Team")
elocombined.to_csv("2024WElos.csv", index = False)

In [ ]:
def get_variance(iqrs, vars, means, teams, df):
    for team in teams:
        Wgames = df[df["WTeamID"] == team]
        Lgames = df[df["LTeamID"] == team]
        changes = pd.concat([Wgames["WChange"], Lgames["LChange"]], axis = 0)
        q1 = changes.quantile(0.25)
        q3 = changes.quantile(0.75)
        iqrs[team] = q3 - q1
        vars[team] = changes.std()
        means[team] = changes.median()
allgames["WTeamID"].unique()

In [ ]:
iqrs = {}
vars = {}
means = {}
get_variance(iqrs, vars, means,allgames["WTeamID"].unique() , allgames)